In [1]:
#Celda para librerías
import sklearn as sk


import matplotlib as plt
import matplotlib.pyplot as plt

import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold

# LightGBM
from lightgbm import LGBMClassifier
import lightgbm as lgb
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
import json

import warnings
warnings.filterwarnings('ignore')

print(np.__version__)
from ydata_profiling import ProfileReport
import os
print("Directorio actual:", os.getcwd())

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 
# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)
# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

#Importando los datos
app_train_def_7 = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/application_train_preprocesado_definitivo_v7.csv')
app_test = pd.read_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/app_test_pruebav1.csv')
print(app_train_def_7.shape)

1.26.4
Directorio actual: /home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS
(307511, 1323)


In [2]:
app_train_pruebas = app_train_def_7.copy()
app_test_pruebas = app_test.copy()

In [3]:
#Columnas cuya varianza es 0 y que por lo tanto son constantes

varianza_cero_cols = [i for i in app_train_pruebas if app_train_pruebas[i].std()==0]
print(varianza_cero_cols)
print(len(varianza_cero_cols))
app_train_pruebas.drop(columns=varianza_cero_cols, inplace=True)
app_test_pruebas.drop(columns=varianza_cero_cols, inplace=True)

['CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MEAN_(BUREAU)', 'CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MAX_(BUREAU)', 'CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MIN_(BUREAU)', 'CLOSED_AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_RATIO_MEAN_(BUREAU)', 'CLOSED_AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_RATIO_MAX_(BUREAU)', 'CLOSED_AMT_CREDIT_SUM_LIMIT_TO_AMT_CREDIT_SUM_OVERDUE_RATIO_MIN_(BUREAU)', 'RATIO_ACTIVE_TO_CLOSED_CREDIT_DAY_OVERDUE_MIN_(BUREAU)', 'RATIO_ACTIVE_TO_CLOSED_CNT_CREDIT_PROLONG_MIN_(BUREAU)', 'RATIO_ACTIVE_TO_CLOSED_AMT_CREDIT_SUM_OVERDUE_MIN_(BUREAU)', 'RATIO_ACTIVE_TO_CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MEAN_(BUREAU)', 'RATIO_ACTIVE_TO_CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MAX_(BUREAU)', 'RATIO_ACTIVE_TO_CLOSED_AMT_CREDIT_SUM_OVERDUE_TO_PROLONGED_RATIO_MIN_(BUREAU)', 'RATIO_ACTIVE_TO_CLOSED_AMT_CREDIT_SUM_TO_AMT_CREDIT_SUM_OVERDUE_RATIO_MEAN_(BUREAU)', 'RATIO_ACTIVE_TO_CLOSED_AMT_CREDIT_SUM_TO_AMT_CREDIT_SUM_OVERDUE_

In [4]:
#Vamos a revisar si en la matriz de correlación tenemos muchas variables con correlación 1 entre sí

corr_matrix = app_train_pruebas.corr()

#Queremos ver solo la parte superior de la matriz pues en la diagonal siempre va a ser 1 pues siempre es correlación = 1 consigo misma dicha variable
upper_side = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = set()
for col in upper_side:
    #Buscamos todas las filas donde la correlación con la variable col sea 1
    corr_perfecta = upper_side.index[upper_side[col] == 1].tolist()
    if len(corr_perfecta) > 0:
        for row in corr_perfecta:
            # row y col tienen correlación 1
            # Decidimos mantener la primera que aparece (row)
            # y marcar 'col' como duplicada a eliminar
            print(f"La columna '{col}' tiene correlación 1 con: {row}. "
                  f"Mantenemos '{row}' y vamos a eliminar '{col}'")
            to_drop.add(col)

print("Columnas candidatas a eliminar por correlación 1:")
print(to_drop)

La columna 'DAYS_OVERDUE_30_DAYS_RATIO_MEAN_(BUREAU)' tiene correlación 1 con: AMT_CREDIT_MAX_OVERDUE_MEAN_(BUREAU). Mantenemos 'AMT_CREDIT_MAX_OVERDUE_MEAN_(BUREAU)' y vamos a eliminar 'DAYS_OVERDUE_30_DAYS_RATIO_MEAN_(BUREAU)'
La columna 'DAYS_OVERDUE_30_DAYS_RATIO_MAX_(BUREAU)' tiene correlación 1 con: AMT_CREDIT_MAX_OVERDUE_MAX_(BUREAU). Mantenemos 'AMT_CREDIT_MAX_OVERDUE_MAX_(BUREAU)' y vamos a eliminar 'DAYS_OVERDUE_30_DAYS_RATIO_MAX_(BUREAU)'
La columna 'DAYS_OVERDUE_30_DAYS_RATIO_MIN_(BUREAU)' tiene correlación 1 con: AMT_CREDIT_MAX_OVERDUE_MIN_(BUREAU). Mantenemos 'AMT_CREDIT_MAX_OVERDUE_MIN_(BUREAU)' y vamos a eliminar 'DAYS_OVERDUE_30_DAYS_RATIO_MIN_(BUREAU)'
La columna 'DAYS_OVERDUE_90_DAYS_RATIO_MIN_(BUREAU)' tiene correlación 1 con: AMT_CREDIT_MAX_OVERDUE_MIN_(BUREAU). Mantenemos 'AMT_CREDIT_MAX_OVERDUE_MIN_(BUREAU)' y vamos a eliminar 'DAYS_OVERDUE_90_DAYS_RATIO_MIN_(BUREAU)'
La columna 'DAYS_OVERDUE_90_DAYS_RATIO_MIN_(BUREAU)' tiene correlación 1 con: DAYS_OVERDUE_30_DA

In [5]:
app_train_pruebas_filtrado = app_train_pruebas.drop(columns=to_drop)
app_test_pruebas_filtrado = app_test_pruebas.drop(columns=to_drop)
print("Tamaño original:", app_train_pruebas.shape, 
      "Tamaño filtrado:", app_train_pruebas_filtrado.shape)
print("Tamaño original:", app_test_pruebas.shape, 
      "Tamaño filtrado:", app_test_pruebas_filtrado.shape)

Tamaño original: (307511, 1295) Tamaño filtrado: (307511, 1156)
Tamaño original: (48744, 1295) Tamaño filtrado: (48744, 1156)


In [6]:
app_train_pruebas_filtrado.to_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/app_def_v7_menor_dimension.csv')
app_test_pruebas_filtrado.to_csv(r'/home/yeray/TFG-Home-Credit-Default-Risk/JUPYTER_NOTEBOOKS/DATA/app_test_menor_dimension.csv')